In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
DATA_DIR = '../../../In-situ Meas Data/Build Command Data/XYPT Commands/' # Directory of XYPT files

dt = 1E-5 # 10 microseconds

In [41]:
data = pd.read_csv(DATA_DIR + 'T500_3D_Scan_Strategies_fused_layer0002.csv', header=None, names=['X', 'Y', 'P', 'T', 'deprecated'])

In [42]:
trigger_indx = np.where(np.array(data['T']))[0]
X = np.array(data['X'])
Y = np.array(data['Y'])
P = np.array(data['P'])

In [6]:
# Speed
x_vel = np.zeros(X.shape)
x_vel[1:-1] = (X[2:]-X[:-2]) / 2 / dt

y_vel = np.zeros(X.shape)
y_vel[1:-1] = (Y[2:]-Y[:-2]) / 2 / dt

speed = np.sqrt(np.square(x_vel) + np.square(y_vel))

In [7]:
# Direction
vel_dir_x = x_vel / (speed + 1e-8)
vel_dir_y = y_vel / (speed + 1e-8)

In [8]:
# Acceleration
x_accel = np.zeros(X.shape)
x_accel[1:-1] = (x_vel[2:]-x_vel[:-2]) / 2 / dt

y_accel = np.zeros(Y.shape)
y_accel[1:-1] = (y_vel[2:]-y_vel[:-2]) / 2 / dt

accel = np.sqrt(np.square(x_accel) + np.square(y_accel))
accel[np.where(speed[1:] < speed[:-1])[0]+1] = accel[np.where(speed[1:] < speed[:-1])[0]+1]*-1

In [9]:
# Energy Density
en_den = P / (speed + 1e-8)

In [10]:
# Heating/Cooling
heating = np.zeros(X.shape)
heating[np.where(P[1:]>P[:-1])[0]+1] = 1
heating[np.where(P[1:]<P[:-1])[0]+1] = -1

In [126]:
# Turning
turn_threshold = 0.01 # Tune me

turning = np.zeros(X.shape)
turn1 = np.zeros(X.shape)
turn2 = np.zeros(X.shape)

turn1[1:] = np.abs(vel_dir_x[1:]-vel_dir_x[:-1])
turn2[1:] = np.abs(vel_dir_y[1:]-vel_dir_y[:-1])
turning = np.any([(turn1 > turn_threshold), (turn2 > turn_threshold)], axis=0)
turning = turning.astype('int')

array([0, 0, 0, ..., 0, 0, 1])

In [114]:
# Residual Heat
R = 10 # parts are 10 mm x 10 mm
T = 5
max_iters = round(T/dt)

residual_heat = np.zeros(X.shape[0])
for i in tqdm(trigger_indx):
    d_ij = np.sqrt(np.square(X[i]-X[max(0, i-max_iters):i]) + np.square(Y[i]-Y[max(0, i-max_iters):i]))
    t_ij = np.linspace((min(max_iters, i))*dt, dt, min(max_iters, i))
    p_j = P[max(0, i-max_iters):i]
    residual_heat[i] += np.sum(np.square((R-d_ij)/R) * ((T-t_ij)/T) * p_j)

100%|██████████████████████████████████████████████████████████████████████████████| 2927/2927 [00:37<00:00, 79.06it/s]


In [128]:
# Merge together, normalize, and save to an excel file
P_trig = P[trigger_indx]
speed_trig = speed[trigger_indx]
x_dir_trig = vel_dir_x[trigger_indx]
y_dir_trig = vel_dir_y[trigger_indx]
accel_trig = accel[trigger_indx]
en_den_trig = en_den[trigger_indx]
heating_trig = heating[trigger_indx]
turning_trig = turning[trigger_indx]
residual_heat_trig = residual_heat[trigger_indx]

pp = pd.DataFrame({'P': P_trig, 'speed': speed_trig, 'x_dir': x_dir_trig, 'y_dir': y_dir_trig, 'accel': accel_trig,
                  'ed': en_den_trig, 'heating': heating_trig, 'turning': turning_trig, 'res_heat': residual_heat_trig})



# pp_normalized = pd.DataFrame()


In [ ]:
# Merge with the labels excel files
label = pd.read_excel()


### Now cycle through each layer 

In [4]:
for ii in tqdm(range(2,251)):
    if ii < 10:
        layer_text = 'layer000' + str(ii)
    elif ii < 100:
        layer_text = 'layer00' + str(ii)
    else:
        layer_text = 'layer0' + str(ii)
        
    data = pd.read_csv(DATA_DIR + 'T500_3D_Scan_Strategies_fused_' + layer_text + '.csv', header=None, 
                       names=['X', 'Y', 'P', 'T', 'deprecated'])
    
    trigger_indx = np.where(np.array(data['T']))[0]
    X = np.array(data['X'])
    Y = np.array(data['Y'])
    P = np.array(data['P'])
    
    # Speed
    x_vel = np.zeros(X.shape)
    x_vel[1:-1] = (X[2:]-X[:-2]) / 2 / dt

    y_vel = np.zeros(X.shape)
    y_vel[1:-1] = (Y[2:]-Y[:-2]) / 2 / dt

    speed = np.sqrt(np.square(x_vel) + np.square(y_vel))
    
    # Direction
    vel_dir_x = x_vel / (speed + 1e-8)
    vel_dir_y = y_vel / (speed + 1e-8)
    
    # Acceleration
    x_accel = np.zeros(X.shape)
    x_accel[1:-1] = (x_vel[2:]-x_vel[:-2]) / 2 / dt

    y_accel = np.zeros(Y.shape)
    y_accel[1:-1] = (y_vel[2:]-y_vel[:-2]) / 2 / dt

    accel = np.sqrt(np.square(x_accel) + np.square(y_accel))
    accel[np.where(speed[1:] < speed[:-1])[0]+1] = accel[np.where(speed[1:] < speed[:-1])[0]+1]*-1
    
    # Energy Density
    en_den = P / (speed + 1e-8)
    
    # Heating/Cooling
    heating = np.zeros(X.shape)
    heating[np.where(P[1:]>P[:-1])[0]+1] = 1
    heating[np.where(P[1:]<P[:-1])[0]+1] = -1
    
    # Turning
    turn_threshold = 0.01 # Tune me

    turning = np.zeros(X.shape)
    turn1 = np.zeros(X.shape)
    turn2 = np.zeros(X.shape)

    turn1[1:] = np.abs(vel_dir_x[1:]-vel_dir_x[:-1])
    turn2[1:] = np.abs(vel_dir_y[1:]-vel_dir_y[:-1])
    turning = np.any([(turn1 > turn_threshold), (turn2 > turn_threshold)], axis=0)
    turning = turning.astype('int')
    
    # Residual Heat
    R = 10 # parts are 10 mm x 10 mm
    T = 5
    max_iters = round(T/dt)

    residual_heat = np.zeros(X.shape[0])
    for i in trigger_indx:
        d_ij = np.sqrt(np.square(X[i]-X[max(0, i-max_iters):i]) + np.square(Y[i]-Y[max(0, i-max_iters):i]))
        t_ij = np.linspace((min(max_iters, i))*dt, dt, min(max_iters, i))
        p_j = P[max(0, i-max_iters):i]
        residual_heat[i] += np.sum(np.square((R-d_ij)/R) * ((T-t_ij)/T) * p_j)
        
    P_trig = P[trigger_indx]
    speed_trig = speed[trigger_indx]
    x_dir_trig = vel_dir_x[trigger_indx]
    y_dir_trig = vel_dir_y[trigger_indx]
    accel_trig = accel[trigger_indx]
    en_den_trig = en_den[trigger_indx]
    heating_trig = heating[trigger_indx]
    turning_trig = turning[trigger_indx]
    residual_heat_trig = residual_heat[trigger_indx]

    pp = pd.DataFrame({'P': P_trig, 'speed': speed_trig, 'x_dir': x_dir_trig, 'y_dir': y_dir_trig, 'accel': accel_trig,
                      'ed': en_den_trig, 'heating': heating_trig, 'turning': turning_trig, 'res_heat': residual_heat_trig})
    
    pp.to_excel('process_parameters/by_layer/' + layer_text + '.xlsx',header=False, index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 249/249 [6:22:35<00:00, 92.19s/it]


### Normalize data for mean=0, std=1 

In [16]:
# Get total number of data
num_data = 0
for ii in tqdm(range(2,251)):
    
    if ii < 10:
        layer_text = 'layer000' + str(ii)
    elif ii < 100:
        layer_text = 'layer00' + str(ii)
    else:
        layer_text = 'layer0' + str(ii)
    
    pp = pd.read_excel('process_parameters/by_layer/' + layer_text + '.xlsx', header=None)
    num_data += pp[0].shape[0]
    
print("The total number of data points is: " + str(num_data))

100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [02:26<00:00,  1.69it/s]

The total number of data points is: 1036741


In [19]:
P_total        = np.zeros((num_data,1))
speed_total    = np.zeros((num_data,1))
x_dir_total    = np.zeros((num_data,1))
y_dir_total    = np.zeros((num_data,1))
accel_total    = np.zeros((num_data,1))
ed_total       = np.zeros((num_data,1))
heating_total  = np.zeros((num_data,1))
turning_total  = np.zeros((num_data,1))
res_heat_total = np.zeros((num_data,1))

# Now collect all the data
indx = 0
for ii in tqdm(range(2,251)):
    
    if ii < 10:
        layer_text = 'layer000' + str(ii)
    elif ii < 100:
        layer_text = 'layer00' + str(ii)
    else:
        layer_text = 'layer0' + str(ii)
    
    pp = pd.read_excel('process_parameters/by_layer/' + layer_text + '.xlsx', header=None)
    
    data_in_layer = pp[0].shape[0]
    
    P = np.array(pp[0])
    speed = np.array(pp[1])
    x_dir = np.array(pp[2])
    y_dir = np.array(pp[3])
    accel = np.array(pp[4])
    ed = np.array(pp[5])
    heating = np.array(pp[6])
    turning = np.array(pp[7])
    res_heat = np.array(pp[8])
    
    new_indx = indx+data_in_layer
    
    P_total[indx:new_indx,0]        = P
    speed_total[indx:new_indx,0]    = speed
    x_dir_total[indx:new_indx,0]    = x_dir
    y_dir_total[indx:new_indx,0]    = y_dir
    accel_total[indx:new_indx,0]    = accel
    ed_total[indx:new_indx,0]       = ed
    heating_total[indx:new_indx,0]  = heating
    turning_total[indx:new_indx,0]  = turning
    res_heat_total[indx:new_indx,0] = res_heat
    
    indx = new_indx

100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [02:23<00:00,  1.74it/s]


In [26]:
# Get normalizing coefficients
pp_mean = dict()
pp_std = dict()

pp_mean['P']        = np.mean(P_total)
pp_mean['speed']    = np.mean(speed_total)
pp_mean['x_dir']    = np.mean(x_dir_total)
pp_mean['y_dir']    = np.mean(y_dir_total)
pp_mean['accel']    = np.mean(accel_total)
pp_mean['ed']       = np.mean(ed_total)
pp_mean['heating']  = np.mean(heating_total)
pp_mean['turning']  = np.mean(turning_total)
pp_mean['res_heat'] = np.mean(res_heat_total)

pp_std['P']        = np.std(P_total)
pp_std['speed']    = np.std(speed_total)
pp_std['x_dir']    = np.std(x_dir_total)
pp_std['y_dir']    = np.std(y_dir_total)
pp_std['accel']    = np.std(accel_total)
pp_std['ed']       = np.std(ed_total)
pp_std['heating']  = np.std(heating_total)
pp_std['turning']  = np.std(turning_total)
pp_std['res_heat'] = np.std(res_heat_total)

In [43]:
# Normalize the data
P_norm        = np.squeeze((P_total - pp_mean['P']) / pp_std['P'])
speed_norm    = np.squeeze((speed_total - pp_mean['speed']) / pp_std['speed'])
x_dir_norm    = np.squeeze((x_dir_total - pp_mean['x_dir']) / pp_std['x_dir'])
y_dir_norm    = np.squeeze((y_dir_total - pp_mean['y_dir']) / pp_std['y_dir'])
accel_norm    = np.squeeze((accel_total - pp_mean['accel']) / pp_std['accel'])
ed_norm       = np.squeeze((ed_total - pp_mean['ed']) / pp_std['ed'])
heating_norm  = np.squeeze(heating_total) # boolean values
turning_norm  = np.squeeze(turning_total) # boolean values
res_heat_norm = np.squeeze((res_heat_total - pp_mean['res_heat']) / pp_std['res_heat'])

In [87]:
image_name = list()
for ii in tqdm(range(2,251)):
    if ii < 10:
        layer_text = 'layer000' + str(ii)
    elif ii < 100:
        layer_text = 'layer00' + str(ii)
    else:
        layer_text = 'layer0' + str(ii)
        
    pp = pd.read_excel('process_parameters/by_layer/' + layer_text + '.xlsx', header=None)
    data_in_layer = pp[0].shape[0]
    
    for image_num in range(data_in_layer):
        image_name.append('layer' + str(ii) + '_' + str(image_num+1) + '.png')


100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [02:38<00:00,  1.57it/s]


In [90]:
# Save the normalized process parameters to a spreadsheet
pp_norm = pd.DataFrame({'P': P_norm, 'speed': speed_norm, 'x_dir': x_dir_norm, 'y_dir': y_dir_norm, 'accel': accel_norm,
                      'ed': ed_norm, 'heating': heating_norm, 'turning': turning_norm, 'res_heat': res_heat_norm,
                       'image_name': np.array(image_name)})
pp_norm.to_excel('process_parameters/normalized_pp.xlsx',header=False, index=False)

### Create a unified spreadsheet holding image name, labels, and process parameters 

In [91]:
pp_norm = pd.read_excel('process_parameters/normalized_pp.xlsx', header=None)
train_labels = pd.read_excel('neural_network_data/train_labels.xlsx', header=None)
dev_labels = pd.read_excel('neural_network_data/dev_labels.xlsx', header=None)
test_labels = pd.read_excel('neural_network_data/test_labels.xlsx', header=None)

In [92]:
np.array(image_name)

array(['layer2_1.png', 'layer2_2.png', 'layer2_3.png', ...,
       'layer250_2916.png', 'layer250_2917.png', 'layer250_2918.png'],
      dtype='<U17')

In [93]:
pp_norm

,0,1,2,3,4,5,6,7,8,9
0,-2.160818,1.079890,1.305310,-0.556242,0.030249,-0.027271,1,0,-0.596975,layer2_1.png
1,-2.160818,1.080203,1.305294,-0.556281,0.020722,-0.027271,0,0,-0.589404,layer2_2.png
2,-2.160818,1.080203,1.305294,-0.556281,0.020722,-0.027271,0,0,-0.581411,layer2_3.png
3,-2.160818,1.080203,1.305294,-0.556281,0.020722,-0.027271,0,0,-0.572986,layer2_4.png
4,-2.160818,1.080203,1.305294,-0.556281,0.020722,-0.027271,0,0,-0.564122,layer2_5.png
...,...,...,...,...,...,...,...,...,...,...
1036736,-0.379472,-0.832575,0.007509,1.418268,0.018591,-0.027271,0,0,1.969726,layer250_2914.png
1036737,-0.379472,-0.832336,0.007509,1.418268,0.022852,-0.027271,0,0,1.995181,layer250_2915.png
1036738,-0.379472,-0.832336,0.007509,1.418268,0.022852,-0.027271,0,0,2.020748,layer250_2916.png
1036739,-0.644715,-1.335333,0.007509,1.418268,-0.818664,-0.027271,-1,0,2.045698,layer250_2917.png


In [76]:
image_name_start = test_labels[0][0].find('layer', 80)
image_name = test_labels[0][0][image_name_start:]
underscore = image_name.find('_')
layer_number = int(image_name[5:underscore])
layer_number

14

In [ ]:
image_dir = '../../../In-situ Meas Data/In-situ Meas Data/Melt Pool Camera Preprocessed PNG/'

image_name_list = np.array(pp_norm[9])

# Train set


# dev set


# test set
P_test        = list()
speed_test    = list()
x_dir_test    = list()
y_dir_test    = list()
accel_test    = list()
ed_test       = list()
heating_test  = list()
turning_test  = list()
res_heat_test = list()
image_name_test = list()
label_test = list()

for ii in tqdm(range(test_labels[0].shape[0])):
    image_loc = test_labels[0][ii]
    image_name_start = image_loc.find('layer', 80)
    if image_name_start == -1:
        raise Exception("Couldn't find image name")
        
    image_name = image_loc[image_name_start:]
    
    underscore = image_name.find('_')
    layer_number = int(image_name[5:underscore])
    
    if layer_number != 1 and layer_number != 5: # we do not have the data for layer1 and layer5 for some reason
        image_name_indx = np.where(image_name_list == image_name)[0][0]
        
        P_test.append(pp_norm[0][image_name_indx])
        speed_test.append(pp_norm[1][image_name_indx])
        x_dir_test.append(pp_norm[2][image_name_indx])
        y_dir_test.append(pp_norm[3][image_name_indx])
        accel_test.append(pp_norm[4][image_name_indx])
        ed_test.append(pp_norm[5][image_name_indx])
        heating_test.append(pp_norm[6][image_name_indx])
        turning_test.append(pp_norm[7][image_name_indx])
        res_heat_test.append(pp_norm[8][image_name_indx])
        image_name_test.append(image_name)
        label_test.append(test_labels[1][ii])

test_labels_pp = pd.DataFrame({'image_name': np.array(image_name_test), 'label': np.array(label_test),
                               'P': np.array(P_test), 'speed': np.array(speed_test), 'x_dir': np.array(x_dir_test), 
                               'y_dir': np.array(y_dir_test), 'accel': np.array(accel_test), 'ed': np.array(ed_test), 
                               'heating': np.array(heating_test), 'turning': np.array(turning_test), 
                               'res_heat': np.array(res_heat_test)})
                                     


  1%|▉                                                                         | 1230/103362 [00:59<1:21:45, 20.82it/s]

In [101]:
pp_norm[0]

0         -2.160818
1         -2.160818
2         -2.160818
3         -2.160818
4         -2.160818
             ...   
1036736   -0.379472
1036737   -0.379472
1036738   -0.379472
1036739   -0.644715
1036740   -2.862527
Name: 0, Length: 1036741, dtype: float64

In [104]:
image_name_indx[0][0]

51484

In [97]:
np.where(image_name_list == 'layer2_255.png')

(array([254], dtype=int64),)

In [98]:
test_labels

,0,1
0,../../../In-situ Meas Data/In-situ Meas Data/M...,18
1,../../../In-situ Meas Data/In-situ Meas Data/M...,9
2,../../../In-situ Meas Data/In-situ Meas Data/M...,14
3,../../../In-situ Meas Data/In-situ Meas Data/M...,3
4,../../../In-situ Meas Data/In-situ Meas Data/M...,0
...,...,...
103357,../../../In-situ Meas Data/In-situ Meas Data/M...,12
103358,../../../In-situ Meas Data/In-situ Meas Data/M...,21
103359,../../../In-situ Meas Data/In-situ Meas Data/M...,22
103360,../../../In-situ Meas Data/In-situ Meas Data/M...,12


In [99]:
pp_norm[0][0]

-2.160818162731815